## Imports and set up experiment specific information

In [ ]:
import tables
import h5py
import numpy as np
import sys
import itertools
import holoviews as hv
hv.extension('bokeh')

expname='xcsx31116'
dirname='/reg/d/psdm/%s/%s/hdf5/smalldata/'%(expname[0:3],expname)

pedSlice = slice(0,200)
pk1FromFullSlice = slice(3145,3175) 
pk1Slice = slice(40,75) 
pk2Slice = slice(75,120) 

runRangeMin = 90
runRangeMax = 107 #run 110 has different timing.

def fileName_forTraces(expname, run):
    return'sml_uxi_pedSub_%s_%04d_new.h5'%(expname, run)
    #return'%s_Run%03d.h5'%(expname, run)

### Plot Example Traces from run 90, 108 & 110
Note the different timing of the second peak for run 90 versus run 110.

In [ ]:
f90=fileName_forTraces(expname, 90)
t90 = tables.open_file('%s/%s'%(dirname, f90)).root.acq01.data.read().squeeze()
f108=fileName_forTraces(expname, 108)
t108 = tables.open_file('%s/%s'%(dirname, f108)).root.acq01.data.read().squeeze()
f110=fileName_forTraces(expname, 110)
t110 = tables.open_file('%s/%s'%(dirname, f110)).root.acq01.data.read().squeeze()


hv.Curve(t90[0],('time','time [arb unit]'),('Volt','Volt')).redim.range(time=(3120, 3300)).options(width=600) * hv.Points(t90[0]).redim.range(x=(3120, 3300)) * \
hv.Curve(t108[2],('time','time [arb unit]')).redim.range(time=(3120, 3300)) * hv.Points(t108[2]).redim.range(x=(3120, 3300)) *\
hv.Curve(t110[10],('time','time [arb unit]')).redim.range(time=(3120, 3300)) * hv.Points(t110[10]).redim.range(x=(3120, 3300)) 

In [ ]:
def getTraces(run):
    fname=fileName_forTraces(expname, run)
    Dat = tables.open_file('%s/%s'%(dirname, fname)).root
    #get the trace
    acqTraces = Dat.acq01.data.read().squeeze()
    #now subtract the pedestal
    acqTracesPed = np.array([trace-np.nanmedian(trace[pedSlice]) for trace in acqTraces])
    #get the position of the first peak and align traces.
    acqPk1Pos = np.array([np.argmax(trace[pk1FromFullSlice]) for trace in acqTracesPed])
    acqPedShift = np.array([trace[(3100+ap1):(3400+ap1)] for trace,ap1 in zip(acqTracesPed,acqPk1Pos) if np.abs(ap1)<10])
    return acqPedShift

traces=[]
traceRuns=[]
for run in np.arange(runRangeMin,runRangeMax):
    try:
        ltraces = getTraces(run)
        for trace in ltraces:
            traces.append(trace)
            traceRuns.append(run)
    except:
        pass

In [ ]:
evts=[0,10,-1]
hv.Curve(traces[evts[0]],('time','time [arb unit]'),('Volt','Volt')).options(width=600) * hv.Points(traces[evts[0]]) *\
hv.Curve(traces[evts[1]],('time','time [arb unit]'),('Volt','Volt')).options(width=600) * hv.Points(traces[evts[1]]) *\
hv.Curve(traces[evts[2]],('time','time [arb unit]'),('Volt','Volt')).options(width=600) * hv.Points(traces[evts[2]])

## Reject saturated traces

In [ ]:
nSatTraces=[]
nSatRuns=[]
for trace, run in zip(traces, traceRuns):
    maxTrace = np.nanmax(trace)
    pixAtMax = (trace>maxTrace*0.99).sum()
    if pixAtMax == 1:
        nSatTraces.append(trace)
        nSatRuns.append(run)

## For flexibility in relative peak position, select single peak as template. 
Need to use second peak to get long tail w/o having to worry that second peak messes with the shape.
Requite decent intensity in second peak while rejecting events that have a lot of intensity in first peak 

In [ ]:
minPk2 = 0.03 #absolute in V
minPk2 = 0.05 #absolute in V - also works, 0.04 does not look good....
maxPk1 = 0.01 #absolute in V
maxBase = 0.0008
#maxBase = 0.0009
#minDiff = 0.05 #relative fraction
pk2Traces=[]
pk2Runs=[]
for trace, run in zip(nSatTraces, nSatRuns):
    #save relative peak height (max) and height of next  reading.
    maxIdx1 = np.argmax(trace[pk1Slice])
    maxIdx2 = np.argmax(trace[pk2Slice])
    acqPk1=trace[pk1Slice][maxIdx1]
    acqPk2=trace[pk2Slice][maxIdx2]
    basePk2=np.array(trace[68:74]).mean()
    #if acqPk2 > minPk2 and acqPk1 < maxPk1:
    if acqPk2 > minPk2 and basePk2 < maxBase:
        pk2Traces.append(trace)
        pk2Runs.append(run)

## Compare sum to single trace
Will use single peak that is "peaky". Sum washes shape out!
Make plot where you can handtune the amount of a sum of two peaks, shifter by one.

In [ ]:
minDiff = 0.08 #relative fraction
pk2PkTraces=[]
pk2PkRuns=[]
for it, trace, run in itertools.izip(itertools.count(), pk2Traces, pk2Runs):
    #save relative peak height (max) and height of next  reading.
    maxIdx2 = np.argmax(trace[pk2Slice])
    acqPk2=trace[pk2Slice][maxIdx2]
    nbrPk1= 1.-max(trace[pk2Slice][maxIdx2-1], trace[pk2Slice][maxIdx2+1])/acqPk2
    #print acqPk2, nbrPk1
    if nbrPk1 > minDiff:
        print len(pk2PkTraces), acqPk2, nbrPk1, np.array(trace[68:74]).mean(), trace[pk2Slice][0], run
        pk2PkTraces.append(trace)
        pk2PkRuns.append(run)

In [ ]:
sumTracePk = np.array(pk2PkTraces).sum(axis=0)#[40:110]
sumTracePk/=sumTracePk.max()
sumTrace = np.array(pk2Traces).sum(axis=0)#[40:110]
sumTrace/=sumTrace.max()
traceInset_sum = sumTrace.copy()[74:89]
traceInset_sumPk = sumTracePk.copy()[74:89]

#pick 0&5 for minPk2=0.05
#pick ..&10
if minPk2==0.05:
    singleIdx=[0,5]
elif minPk2==0.04:
    singleIdx=[0,12]
else:
    singleIdx=[9,13]
    
singleTrace = pk2PkTraces[singleIdx[0]]#[40:110]
singleTrace/=singleTrace.max()
traceInset_single = singleTrace.copy()[74:89]
singleTrace2 = pk2PkTraces[singleIdx[1]]#[40:110]
singleTrace2/=singleTrace2.max()
traceInset_single2 = singleTrace2.copy()[74:89]

t1=(40,150)
plot1=\
hv.Curve(sumTrace,('time','time [arb unit]'),('Volt','Volt')).redim.range(time=t1).options(width=600) *\
hv.Curve(sumTracePk,('time','time [arb unit]'),('Volt','Volt')).redim.range(time=t1).options(width=600) *\
hv.Curve(singleTrace,('time','time [arb unit]'),('Volt','Volt')).redim.range(time=t1).options(width=600) *\
hv.Curve(singleTrace2,('time','time [arb unit]'),('Volt','Volt')).redim.range(time=t1).options(width=600) *\
hv.Points(sumTrace, kdims=['time','Volt']).redim.range(time=t1) *\
hv.Points(sumTracePk, kdims=['time','Volt']).redim.range(time=t1) *\
hv.Points(singleTrace, kdims=['time','Volt']).redim.range(time=t1) *\
hv.Points(singleTrace2, kdims=['time','Volt']).redim.range(time=t1)

plot2=\
hv.Curve(traceInset_sum,('time2','Time [arb unit]'),('volt','volt')).options(width=300) *\
hv.Curve(traceInset_sumPk,('time2','Time [arb unit]'),('Volt2','Volt')).options(width=300) *\
hv.Curve(traceInset_single,('time2','Time [arb unit]'),('Volt2','Volt')).options(width=300) *\
hv.Curve(traceInset_single2,('time2','Time [arb unit]'),('Volt2','Volt')).options(width=300)
plot1+plot2

## Save template traces to hdf5 file.

In [ ]:
write=True
if write:
    with h5py.File("SingleAcqirisPeak_notebook.h5", "w") as f:
        f['sumTrace']=sumTrace
        f['sumTracePk']=sumTracePk
        f['singlePicked']=singleTrace
        f['singlePicked_alt']=singleTrace2
        f['acq_pk2_singles']=np.array([pk2/max(pk2) for pk2 in pk2Traces])
        f['acq_pk2_singleRuns']=pk2Runs
        f.close()